In [28]:
BATCH_SIZE = 24
IMAGE_SIZE = 64
SEED = 42
AUTOENCODER_EPOCHS = 30
CLASSIFIER_EPOCHS = 10
num_classes = 5 
class_names = ["cardboard", "glass", "metal", "paper", "plastic"]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Rescaling

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from scikeras.wrappers import KerasClassifier
import tensorflow_datasets as tfds



In [29]:
dataset_dir = "./dataset"
train, validate = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=SEED,
    subset="both",
    validation_split=0.2,
    interpolation='bilinear',

)


Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Using 478 files for validation.


In [30]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
flip_layer = tf.keras.layers.RandomFlip("horizontal_and_vertical")
rotation_layer = tf.keras.layers.RandomRotation(0.2)
def change_inputs(images, labels):
    x = flip_layer(images)
    x = rotation_layer(x)

    x = tf.image.resize(x, [IMAGE_SIZE, IMAGE_SIZE], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return x, x

normalized_ds = train.map(change_inputs)

# Autoencoder Model
def create_autoencoder_model():
    input_layer = tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = tf.keras.layers.Dropout(0.5)(encoded) 
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    # x = tf.keras.layers.Dropout(0.5)(encoded) 
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse', )
    return autoencoder

# Training Autoencoder
autoencoder_model = create_autoencoder_model()
history_autoencoder = autoencoder_model.fit(normalized_ds, epochs=AUTOENCODER_EPOCHS)

# Encode data using the trained autoencoder
encoded_data = autoencoder_model.predict(normalized_ds)
print(f"Encoded Data Shape: {encoded_data.shape}")





Epoch 1/10
60/60 [==============================] - 30s 467ms/step - loss: 4608.1685
Epoch 2/10
60/60 [==============================] - 29s 479ms/step - loss: 669.3286
Epoch 3/10
60/60 [==============================] - 29s 477ms/step - loss: 459.4630
Epoch 4/10
60/60 [==============================] - 29s 480ms/step - loss: 392.5450
Epoch 5/10
60/60 [==============================] - 30s 490ms/step - loss: 356.9963
Epoch 6/10
60/60 [==============================] - 29s 484ms/step - loss: 328.2575
Epoch 7/10
60/60 [==============================] - 29s 482ms/step - loss: 319.2357
Epoch 8/10
60/60 [==============================] - 29s 482ms/step - loss: 303.3122
Epoch 9/10
60/60 [==============================] - 29s 481ms/step - loss: 295.0629
Epoch 10/10
60/60 [==============================] - 9s 142ms/step
Encoded Data Shape: (1912, 64, 64, 3)


In [ ]:
def create_classifier_model(autoencoder_model):
    # Use the encoder part of the autoencoder as a feature extractor
    encoder_output = autoencoder_model.layers[7].output  # Assuming the encoder output is at index 7, adjust if needed

    # Add classification layers on top of the encoder output
    x = layers.Flatten()(encoder_output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    classifier_model = Model(autoencoder_model.input, output_layer)
    classifier_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return classifier_model

# Pass your autoencoder model when creating the classifier
classifier_model = create_classifier_model(autoencoder_model)
history_classifier = classifier_model.fit(train, epochs=CLASSIFIER_EPOCHS, validation_data=validate)

In [ ]:
validate_normalized = validate.map(change_inputs)
validate_list = list(validate_normalized.as_numpy_iterator())
images_and_labels = list(validate.as_numpy_iterator())
# Predict labels for validation images
predictions = classifier_model.predict(validate)
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)
y = np.concatenate([y for x, y in validate], axis=0)

# Display a few validation images along with their true and predicted labels
plt.figure(figsize=(10, 10))
for i in range(4):  # Adjust the number of images you want to display
    plt.subplot(4, 4, i + 1)
    
    # Access the batch of images and labels
    image_batch = images_and_labels[i][0]
    label_batch = images_and_labels[i][1]
    
    # Access the ith image and label from the batch
    image = image_batch[i]
    label = label_batch[i]
    
    plt.imshow(image.astype(np.uint8) / 255)
    plt.title(f"True: {class_names[label]}\nPredicted: {class_names[predicted_labels[i]]}")
    plt.axis('off')

plt.show()


In [ ]:
normalized_validate = validate.map(change_inputs)
evaluation_results = autoencoder.evaluate(normalized_validate)
print("Validation Loss:", evaluation_results)


In [ ]:
def get_validation_batch(dataset, batch_size=5):
    for images, _ in dataset.take(1):
        return images[:batch_size]

# Get a batch of images
sample_images = get_validation_batch(normalized_validate)

# Generate reconstructions
reconstructed_images = autoencoder.predict(sample_images)

# Visualize original vs. reconstructed images
for i in range(len(sample_images)):
    plt.subplot(2, len(sample_images), i + 1)
    plt.imshow(sample_images[i])
    plt.title("Original")
    plt.axis("off")

    plt.subplot(2, len(sample_images), i + 1 + len(sample_images))
    plt.imshow(reconstructed_images[i])
    plt.title("Recon")
    plt.axis("off")

plt.show()